## Notes

In [3]:
# Environment Setup
library(tidyverse)
library(tidymodels)
library(readxl)

options(repr.plot.width = 10, repr.plot.height = 8)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

── Attaching packages ────────────────────────────────────── tidymodels 1.0.0 ──

✔ broom        1.0.0     ✔ rsample      1.0.0
✔ dials        1.0.0     ✔ tune         1.0.0
✔ infer        1.0.2     ✔ workflows    1.0.0
✔ modeldata    1.0.0     ✔ workflowsets 1.0.0
✔ parsnip      1.0.0     ✔ yardstick    1.0.0
✔ recipes      1.0.1     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filter()   masks stats::filter()
✖ recipes::fixed()  masks stringr::fixed()
✖ dplyr::lag()      masks stats::lag()
✖ yardstick::spec() masks r

# Credit Card Default Prediction
___

## Introduction
___

**TODO: (!!! delete below in final draft)**
* Provide some relevant background information on the topic so that someone unfamiliar with it will be prepared to understand the rest of your proposal
* Clearly state the question you will try to answer with your project
* Identify and describe the dataset that will be used to answer the question

#### **Background** ####
Credit Card is an essential part of our daily lives today. Per Statista there were 76M credit cards in circulation within Canada implaying 2 per every Canadian. In addition to the impact it has on everyday citizens, credit card is booming and loan securitization is a huge business for banks and asset managers. As such, a large credit card default at a macro level could lead to systemic failures of banks and the broader capital markets similar to the one observed in GFC.

#### **Thesis** ####
Using the data and the techniques learned in class, we would like to answer the question: **can we predict the default status of a credit card client?**

#### **Data** ####
The data is from an unnamed debit and credit card issuing bank in Taiwan. The data consists of information about 30,000 customers as at October 2005, of which 23,364 (78%) have not defaulted while 6,636 (22%) have defaulted. The default status is represented as binary variable (1 = Yes, 0 = No). Variables available to be used as direct inputs are as below:
* **X1**: Amount of Credit Given (NT$)
* **X2**: Sex (1 = Male, 2 = Female)
* **X3**: Education (1 = Graduate School, 2 = University, 3 = High School, 4 = Others)
* **X4**: Marital Status (1 = Married, 2 = Single, 3 = Others)
* **X5**: Age (# of Years)
* **X6-X11**: Repayment Status (-1 = Clear, 1 = Payment Delay of 1 Month, 2 = Payment Delay of 2 Months, ..., 8 = Payment Delay of 8 Months, 9 = Payment Delay of 9 Months or greater)
    * **X6**  = Repayment status in 2005-09
    * **X7**  = Repayment status in 2005-08
    * **X8**  = Repayment status in 2005-07
    * **X9**  = Repayment status in 2005-06
    * **X10** = Repayment status in 2005-05
    * **X11** = Repayment status in 2005-04
* **X12-X17**: Amount of Bill Statement (NT$)
    * **X12** = Amount of Bill Statement in 2005-09
    * **X13** = Amount of Bill Statement in 2005-08
    * **X14** = Amount of Bill Statement in 2005-07
    * **X15** = Amount of Bill Statement in 2005-06
    * **X16** = Amount of Bill Statement in 2005-05
    * **X17** = Amount of Bill Statement in 2005-04
* **X18-X23**: Amount of Previous Payment (NT$)
    * **X18** = Amount Paid in 2005-09
    * **X19** = Amount Paid in 2005-08
    * **X21** = Amount Paid in 2005-06
    * **X20** = Amount Paid in 2005-07
    * **X21** = Amount Paid in 2005-05
    * **X23** = Amount Paid in 2005-04

Source of Data: https://archive.ics.uci.edu/ml/datasets/default+of+credit+card+clients

## Preliminary Exploratory Data Analysis
___

* Demonstrate that the dataset can be read from the web into R 
* Clean and wrangle your data into a tidy format
* Using only **training data**, summarize the data in at least one table (this is exploratory data analysis). An example of a useful table could be one that reports the number of observations in each class, the means of the predictor variables you plan to use in your analysis and how many rows have missing data. 
* Using only **training data**, visualize the data with at least one plot relevant to the analysis you plan to do (this is exploratory data analysis). An example of a useful visualization could be one that compares the distributions of each of the predictor variables you plan to use in your analysis.


In [4]:
columns = c('id', 'credit_limit', 'sex', 'education', 'marital_status', 'age',
            'status_09', 'status_08', 'status_07', 'status_06', 'status_05', 'status_04',
            'balance_09', 'balance_08', 'balance_07', 'balance_06', 'balance_05', 'balance_04',
            'payment_09', 'payment_08', 'payment_07', 'payment_06', 'payment_05', 'payment_04',
            'y')
            
credit_card_data <- read_excel('data/default of credit card clients.xls',sheet='Data',skip=2,col_names=columns) |>
    select(-id)

head(credit_card_data)
glimpse(credit_card_data)

credit_limit,sex,education,marital_status,age,status_09,status_08,status_07,status_06,status_05,⋯,balance_06,balance_05,balance_04,payment_09,payment_08,payment_07,payment_06,payment_05,payment_04,y
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
20000,2,2,1,24,2,2,-1,-1,-2,⋯,0,0,0,0,689,0,0,0,0,1
120000,2,2,2,26,-1,2,0,0,0,⋯,3272,3455,3261,0,1000,1000,1000,0,2000,1
90000,2,2,2,34,0,0,0,0,0,⋯,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
50000,2,2,1,37,0,0,0,0,0,⋯,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
50000,1,2,1,57,-1,0,-1,0,0,⋯,20940,19146,19131,2000,36681,10000,9000,689,679,0
50000,1,1,2,37,0,0,0,0,0,⋯,19394,19619,20024,2500,1815,657,1000,1000,800,0


Rows: 30,000
Columns: 24
$ credit_limit   <dbl> 20000, 120000, 90000, 50000, 50000, 50000, 500000, 1000…
$ sex            <dbl> 2, 2, 2, 2, 1, 1, 1, 2, 2, 1, 2, 2, 2, 1, 1, 2, 1, 1, 2…
$ education      <dbl> 2, 2, 2, 2, 2, 1, 1, 2, 3, 3, 3, 1, 2, 2, 1, 3, 1, 1, 1…
$ marital_status <dbl> 1, 2, 2, 1, 1, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 3, 2, 1, 1…
$ age            <dbl> 24, 26, 34, 37, 57, 37, 29, 23, 28, 35, 34, 51, 41, 30,…
$ status_09      <dbl> 2, -1, 0, 0, -1, 0, 0, 0, 0, -2, 0, -1, -1, 1, 0, 1, 0,…
$ status_08      <dbl> 2, 2, 0, 0, 0, 0, 0, -1, 0, -2, 0, -1, 0, 2, 0, 2, 0, 0…
$ status_07      <dbl> -1, 0, 0, 0, -1, 0, 0, -1, 2, -2, 2, -1, -1, 2, 0, 0, 2…
$ status_06      <dbl> -1, 0, 0, 0, 0, 0, 0, 0, 0, -2, 0, -1, -1, 0, 0, 0, 2, …
$ status_05      <dbl> -2, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, -1, -1, 0, 0, 0, 2, …
$ status_04      <dbl> -2, 2, 0, 0, 0, 0, 0, -1, 0, -1, -1, 2, -1, 2, 0, 0, 2,…
$ balance_09     <dbl> 3913, 2682, 29239, 46990, 8617, 64400, 367965, 11876, 1…
$ balance_08   

In [46]:
### set.seed(154)
credit_card_split <- credit_card_data |> initial_split(prop=0.75, strata = y)
credit_card_train <- training(credit_card_split)
credit_card_test <- testing(credit_card_split)


In [47]:
credit_card_ave_table <- credit_card_train |> 
        filter(balance_09 != is.na(balance_09))|>
        filter(balance_08 != is.na(balance_08))|>
        filter(balance_07 != is.na(balance_07))|>
        filter(balance_06 != is.na(balance_06))|>
        filter(balance_05 != is.na(balance_05))|>
        filter(balance_04 != is.na(balance_04))|>
        mutate(balance_ave = (balance_09 + balance_08 + balance_07 + balance_06 + balance_05 + balance_04)/3)|>
        filter(payment_09 != is.na(payment_09))|>
        filter(payment_08 != is.na(payment_08))|>
        filter(balance_07 != is.na(balance_07))|>
        filter(payment_06 != is.na(payment_06))|>
        filter(payment_05 != is.na(payment_05))|>
        filter(balance_04 != is.na(balance_04))|>
        mutate(payment_ave = (payment_09 + payment_08 + payment_07 + payment_06 + payment_05 + payment_04)/6)|>
        select(y, credit_limit, age, payment_ave, balance_ave)|> 
        group_by(y)|>
        summarize(credit_limit_ave = mean(credit_limit, na.rm = TRUE), 
                  age_ave = mean(age, na.rm = TRUE), 
                  payment_ave = mean(payment_ave, na.rm = TRUE), 
                  balance_ave = mean(balance_ave, na.rm = TRUE))


credit_card_ave_table

y,credit_limit_ave,age_ave,payment_ave,balance_ave
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0,186410.8,35.49840,7262.180,120457.0
1,137162.1,35.88201,4937.463,125572.3


In [48]:
credit_card_status_0 <- credit_card_train |>
        filter(y == 0)|>
        select(y, status_09, status_08, status_07, status_06, status_05, status_04)|>
        group_by(y, status_09)|>
        summarize(n = n())

credit_card_status_0

credit_card_status_1 <- credit_card_train |>
        filter(y == 1)|>
        select(y, status_09, status_08, status_07, status_06, status_05, status_04)|>
        group_by(y, status_09)|>
        summarize(n = n())

credit_card_status_1

`summarise()` has grouped output by 'y'. You can override using the `.groups`
argument.


y,status_09,n
<dbl>,<dbl>,<int>
0,-2,1802
0,-1,3541
0,0,9601
0,1,1864
0,2,617
0,3,57
0,4,21
0,5,8
0,6,3


`summarise()` has grouped output by 'y'. You can override using the `.groups`
argument.


y,status_09,n
<dbl>,<dbl>,<int>
1,-2,274
1,-1,718
1,0,1433
1,1,933
1,2,1363
1,3,187
1,4,38
1,5,10
1,6,6


In [49]:
credit_card_max_min_table <- credit_card_train|>
        select(-sex, -education, -marital_status, -status_09, -status_08, -status_07, -status_06, -status_05, -status_04)|>
        group_by(y)|>
        summarize(max_age = max(age), 
                  min_age = min(age), 
                  max_balance_09 = max(balance_09),
                  min_balance_09 = min(balance_09),
                  max_balance_08 = max(balance_08),
                  min_balance_08 = min(balance_08),
                  max_balance_07 = max(balance_07),
                  min_balance_07 = min(balance_07),
                  max_balance_06 = max(balance_06),
                  min_balance_06 = min(balance_06),
                  max_balance_05 = max(balance_05),
                  min_balance_05 = min(balance_05),
                  max_balance_04 = max(balance_04),
                  min_balance_04 = min(balance_04))


credit_card_max_min_table

y,max_age,min_age,max_balance_09,min_balance_09,max_balance_08,min_balance_08,max_balance_07,min_balance_07,max_balance_06,min_balance_06,max_balance_05,min_balance_05,max_balance_04,min_balance_04
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0,79,21,964511,-154973,983931,-67526,1664089,-157264,891586,-170000,927171,-81334,961664,-209051
1,75,21,610723,-6676,581775,-17710,578971,-61506,548020,-50616,547880,-53007,498316,-339603


## Methods
___

* Explain how you will conduct either your data analysis and which variables/columns you will use. <u>Note - you do not need to use all variables/columns that exist in the raw data set. In fact, that's often not a good idea. For each variable think: is this a useful variable for prediction?</u>
* Describe at least one way that you will visualize the results

## Expected Outcomes and Significance
___

* What do you expect to find?
* What impact could such findings have?
* What future questions could this lead to?

Using this data and applying the information we learned in this course, we plan to predict the default status of a credit card client using Machine Learning.

With this data and predictions, we can:

1: Create a less risky credit environment for both banks and people
2: Provide better rates for less-risky credit card owners
3: Develop educational programs for people in higher risk levels for better credit card management.

This could lead to questions like: How can we increase efficiency as well as create a low-risk environment where everyone can afford to have a credit card?